# BigQuery ML models with feature engineering

In this notebook, we will use BigQuery ML to build more sophisticated models for taxifare prediction.

This is a continuation of our [first models](../02_bqml/first_model.ipynb)

In [ ]:
import os
PROJECT = "qwiklabs-gcp-bdc77450c97b4bf6" # REPLACE WITH YOUR PROJECT ID
BUCKET = "your-bucket-id-here" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

In [ ]:
# if you have not already done so ...
%%bash
bq --location=US mk --dataset \
    --description 'Taxi Fare' \
    $PROJECT:serverlessml

## Model 4: With some transformations

BigQuery ML automatically scales the inputs. so we don't need to do scaling, but human insight can help.

Since we we'll repeat this quite a bit, let's make a dataset with 1 million rows. 

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE serverlessml.feateng_training_data AS

SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_datetime,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM `nyc-tlc.yellow.trips`
WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 1000) = 1
AND
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL serverlessml.model4_feateng
TRANSFORM(
  * EXCEPT(pickup_datetime)
  , ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean
  , CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING) AS dayofweek
  , CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING) AS hourofday
)
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg') 
AS

SELECT * FROM serverlessml.feateng_training_data

Once the training is done, visit the [BigQuery Cloud Console](https://console.cloud.google.com/bigquery) and look at the model that has been trained. Then, come back to this notebook.

Note that BigQuery automatically split the data we gave it, and trained on only a part of the data and used the rest for evaluation. We can look at eval statistics on that held-out data:

In [ ]:
%%bigquery
SELECT *, SQRT(loss) AS rmse FROM ML.TRAINING_INFO(MODEL serverlessml.model4_feateng)

In [ ]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model4_feateng)

Yippee! We're now below our target of 6 dollars in RMSE.
We are now beating our goals, and with just a linear model.  This is how the prediction query would look:

In [ ]:
%%bigquery
SELECT * FROM ML.PREDICT(MODEL serverlessml.model4_feateng, (
  SELECT 
    -73.982683 AS pickuplon,
    40.742104 AS pickuplat,
    -73.983766 AS dropofflon,
    40.755174 AS dropofflat,
    3.0 AS passengers,
    TIMESTAMP('2019-06-03 04:21:29.769443 UTC') AS pickup_datetime
))

## Let's do a feature cross of the day-hour combination instead of using them raw

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL serverlessml.model5_featcross
TRANSFORM(
  * EXCEPT(pickup_datetime)
  , ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean
  , CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING),
           CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS day_hr
)
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg') 
AS

SELECT * FROM serverlessml.feateng_training_data

In [ ]:
%%bigquery
SELECT *, SQRT(loss) AS rmse FROM ML.TRAINING_INFO(MODEL serverlessml.model5_featcross)

In [ ]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model5_featcross)

Sometimes (not the case above), the training RMSE is quite reasonable, but the evaluation RMSE is terrible. This is an indication of overfitting.
When we do feature crosses, we run into the risk of overfitting (for example, when a particular day-hour combo doesn't have enough taxirides).

## Reducing overfitting

Let's add L2 regularization.

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL serverlessml.model6_featcross_l2
TRANSFORM(
  * EXCEPT(pickup_datetime)
  , ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean
  , CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING),
           CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS day_hr
)
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg', l2_reg=0.1) 
AS

SELECT * FROM serverlessml.feateng_training_data

In [ ]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model6_featcross_l2)

These sorts of experiment would have taken days to do otherwise. We did it in minutes, thanks to BigQuery ML!  The advantage of doing all this in the TRANSFORM is the client code doing the PREDICT doesn't change. Our model improvement is transparent to client code.

In [ ]:
%%bigquery
SELECT * FROM ML.PREDICT(MODEL serverlessml.model6_featcross_l2, (
  SELECT 
    -73.982683 AS pickuplon,
    40.742104 AS pickuplat,
    -73.983766 AS dropofflon,
    40.755174 AS dropofflat,
    3.0 AS passengers,
    TIMESTAMP('2019-06-03 04:21:29.769443 UTC') AS pickup_datetime
))

## Let's try feature crossing the locations too

Because the lat and lon by themselves don't have meaning, but only in conjunction, it may be useful to treat the fields as a pair instead of just using them as numeric values. However, lat and lon are continuous numbers, so we have to discretize them first. That's what SnapToGrid does. 

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL serverlessml.model7_geo
TRANSFORM(
  fare_amount
  , ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean
  , CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING),
           CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS day_hr
  , CONCAT(
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplon, pickuplat), 0.01)),
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropofflon, dropofflat), 0.01))
  ) AS pickup_and_dropoff
)
OPTIONS(input_label_cols=['fare_amount'], model_type='linear_reg', l2_reg=0.1) 
AS

SELECT * FROM serverlessml.feateng_training_data

In [ ]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model7_geo)

Yippee!  We're now below our target of 6 dollars in RMSE.

## DNN

You could, of course, train a more sophisticated model.  Change "linear_reg" above to "dnn_regressor" and see if it improves things.

In [ ]:
%%bigquery
-- This is alpha and may not work for you.
CREATE OR REPLACE MODEL serverlessml.model8_dnn
TRANSFORM(
  fare_amount
  , ST_Distance(ST_GeogPoint(pickuplon, pickuplat), ST_GeogPoint(dropofflon, dropofflat)) AS euclidean
  , CONCAT(CAST(EXTRACT(DAYOFWEEK FROM pickup_datetime) AS STRING),
           CAST(EXTRACT(HOUR FROM pickup_datetime) AS STRING)) AS day_hr
  , CONCAT(
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickuplon, pickuplat), 0.01)),
     ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropofflon, dropofflat), 0.01))
  ) AS pickup_and_dropoff
)
-- at the time of writing, l2_reg wasn't supported yet.
OPTIONS(input_label_cols=['fare_amount'], model_type='dnn_regressor', hidden_units=[32, 8]) 
AS

SELECT * FROM serverlessml.feateng_training_data

In [ ]:
%%bigquery
SELECT SQRT(mean_squared_error) AS rmse FROM ML.EVALUATE(MODEL serverlessml.model8_dnn)

We really need the L2 reg (recall that we got 4.77 without the feateng). Let's do this in Keras.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.